In [1]:
# Install necessary packages
!pip install boto3 pandas

# Import required libraries
import boto3
import pandas as pd
from boto3.dynamodb.conditions import Key

# Initialize DynamoDB resource
dynamodb = boto3.resource('dynamodb', region_name='us-east-2')



In [2]:
# Function to create a DynamoDB table with a secondary index
def create_table():
    try:
        table = dynamodb.create_table(
            TableName='Orders',
            KeySchema=[
                {
                    'AttributeName': 'CustomerID',
                    'KeyType': 'HASH'  # Partition key
                },
                {
                    'AttributeName': 'OrderID',
                    'KeyType': 'RANGE'  # Sort key
                }
            ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'CustomerID',
                    'AttributeType': 'S'
                },
                {
                    'AttributeName': 'OrderID',
                    'AttributeType': 'S'
                },
                {
                    'AttributeName': 'ProductCategory',
                    'AttributeType': 'S'
                },
                {
                    'AttributeName': 'OrderDate',
                    'AttributeType': 'S'
                }
            ],
            ProvisionedThroughput={
                'ReadCapacityUnits': 5,
                'WriteCapacityUnits': 5
            },
            GlobalSecondaryIndexes=[
                {
                    'IndexName': 'ProductCategoryIndex',
                    'KeySchema': [
                        {
                            'AttributeName': 'ProductCategory',
                            'KeyType': 'HASH'
                        },
                        {
                            'AttributeName': 'OrderDate',
                            'KeyType': 'RANGE'
                        }
                    ],
                    'Projection': {
                        'ProjectionType': 'ALL'
                    },
                    'ProvisionedThroughput': {
                        'ReadCapacityUnits': 5,
                        'WriteCapacityUnits': 5
                    }
                }
            ]
        )
        table.meta.client.get_waiter('table_exists').wait(TableName='Orders')
        print("Table created successfully.")
    except Exception as e:
        print(e)



In [3]:

# Function to load data from a CSV file and insert into DynamoDB
def load_data(csv_file):
    table = dynamodb.Table('Orders')
    df = pd.read_csv(csv_file)
    print(df)
    
    for index, row in df.iterrows():
        table.put_item(
            Item={
                'CustomerID': row['CustomerID'],
                'OrderID': row['OrderID'],
                'OrderDate': row['OrderDate'],
                'TotalAmount': str(row['TotalAmount']),  # DynamoDB requires numbers as strings in some cases
                'ProductCategory': row['ProductCategory'],
                'Quantity': int(row['Quantity'])  # Ensure Quantity is an integer
            }
        )
    print("Data loaded successfully.")

In [4]:
# Function to perform a query on the DynamoDB table
def query_data(customer_id):
    table = dynamodb.Table('Orders')
    response = table.query(
        KeyConditionExpression=Key('CustomerID').eq(customer_id)
    )
    return response['Items']

# Function to query using a secondary index
def query_secondary_index(category, start_date, end_date):
    table = dynamodb.Table('Orders')
    response = table.query(
        IndexName='ProductCategoryIndex',
        KeyConditionExpression=Key('ProductCategory').eq(category) & Key('OrderDate').between(start_date, end_date)
    )
    return response['Items']


In [5]:

# Create the table
create_table()

Table created successfully.


In [6]:
# Load data from CSV file
# Assume the CSV file 'orders.csv' has columns: CustomerID, OrderID, OrderDate, TotalAmount, ProductCategory, Quantity
csv_file = './data/orders.csv'
load_data(csv_file)

   CustomerID OrderID   OrderDate  TotalAmount ProductCategory  Quantity
0        C001   O1001  2024-05-27       250.75     Electronics         1
1        C001   O1002  2024-05-28       150.50           Books         3
2        C002   O1003  2024-05-27       175.50        Clothing         2
3        C002   O1004  2024-05-29       200.00     Electronics         1
4        C001   O1005  2024-05-30       300.00       Furniture         1
5        C003   O1006  2024-05-27       100.00           Books         1
6        C003   O1007  2024-05-28        75.00        Clothing         4
7        C001   O1008  2024-05-31        80.00           Books         2
8        C002   O1009  2024-06-01        60.00        Clothing         1
9        C003   O1010  2024-06-01       500.00     Electronics         1
10       C001   O1011  2024-06-02       250.00       Furniture         1
11       C002   O1012  2024-06-02        90.00           Books         3
12       C003   O1013  2024-06-03       400.00     

In [7]:

# Query the data for a specific customer
customer_id = 'C001'
items = query_data(customer_id)
print("Query results for CustomerID = C001:")
for item in items:
    print(item)

Query results for CustomerID = C001:
{'ProductCategory': 'Electronics', 'Quantity': Decimal('1'), 'TotalAmount': '250.75', 'CustomerID': 'C001', 'OrderDate': '2024-05-27', 'OrderID': 'O1001'}
{'ProductCategory': 'Books', 'Quantity': Decimal('3'), 'TotalAmount': '150.5', 'CustomerID': 'C001', 'OrderDate': '2024-05-28', 'OrderID': 'O1002'}
{'ProductCategory': 'Furniture', 'Quantity': Decimal('1'), 'TotalAmount': '300.0', 'CustomerID': 'C001', 'OrderDate': '2024-05-30', 'OrderID': 'O1005'}
{'ProductCategory': 'Books', 'Quantity': Decimal('2'), 'TotalAmount': '80.0', 'CustomerID': 'C001', 'OrderDate': '2024-05-31', 'OrderID': 'O1008'}
{'ProductCategory': 'Furniture', 'Quantity': Decimal('1'), 'TotalAmount': '250.0', 'CustomerID': 'C001', 'OrderDate': '2024-06-02', 'OrderID': 'O1011'}
{'ProductCategory': 'Clothing', 'Quantity': Decimal('3'), 'TotalAmount': '180.0', 'CustomerID': 'C001', 'OrderDate': '2024-06-03', 'OrderID': 'O1014'}
{'ProductCategory': 'Electronics', 'Quantity': Decimal('1'

In [8]:

# Query the data using the secondary index
category = 'Electronics'
start_date = '2024-05-27'
end_date = '2024-06-29'
items = query_secondary_index(category, start_date, end_date)
print(f"\nQuery results for ProductCategory = {category} between {start_date} and {end_date}:")
for item in items:
    print(item)


Query results for ProductCategory = Electronics between 2024-05-27 and 2024-06-29:
{'TotalAmount': '250.75', 'ProductCategory': 'Electronics', 'Quantity': Decimal('1'), 'CustomerID': 'C001', 'OrderID': 'O1001', 'OrderDate': '2024-05-27'}
{'TotalAmount': '200.0', 'ProductCategory': 'Electronics', 'Quantity': Decimal('1'), 'CustomerID': 'C002', 'OrderID': 'O1004', 'OrderDate': '2024-05-29'}
{'TotalAmount': '500.0', 'ProductCategory': 'Electronics', 'Quantity': Decimal('1'), 'CustomerID': 'C003', 'OrderID': 'O1010', 'OrderDate': '2024-06-01'}
{'TotalAmount': '400.0', 'ProductCategory': 'Electronics', 'Quantity': Decimal('2'), 'CustomerID': 'C003', 'OrderID': 'O1013', 'OrderDate': '2024-06-03'}
{'TotalAmount': '300.0', 'ProductCategory': 'Electronics', 'Quantity': Decimal('1'), 'CustomerID': 'C001', 'OrderID': 'O1017', 'OrderDate': '2024-06-05'}
{'TotalAmount': '225.0', 'ProductCategory': 'Electronics', 'Quantity': Decimal('1'), 'CustomerID': 'C002', 'OrderID': 'O1021', 'OrderDate': '2024